In [50]:
import os
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git
import datetime
import time

In [89]:
import numpy as np
import pandas as pd
import os
import re
import copy
from collections import Counter
# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
from collections import Counter
import git
from git import RemoteProgress
import pprint as pprint
from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from collections import Counter
%matplotlib inline

In [90]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [91]:
repo = Repo("camel-master/")
fixing_commit ="2c6964ae94d8f9a9c9a32e5ae5a0b794e8b8d3b"

In [162]:
show_info = repo.git.show("-s", fixing_commit).splitlines()
for line in show_info:
    print(line)

commit 9fd5cb7dfe0fcc431f99d5e14206e0090e72f36b
Author: Dejan Bosanac <dejan@nighttale.net>
Date:   Thu Apr 30 12:08:26 2015 +0200

    https://issues.apache.org/jira/browse/AMQ-5754 - disable file server by default


In [163]:
online_link_camel= 'https://github.com/apache/camel'
online_link_activemq = 'https://github.com/apache/activemq'
online_link_restlet_framework= 'https://github.com/restlet/restlet-framework-java'


In [164]:
local_link_camel= "camel-master/"
local_link_activemq= "activemq/"
local_link_restlet_framework = "restlet-framework-java/"


In [165]:
camel_fixing_commit ='2c6964ae94d8f9a9c9a32e5ae5a0b794e8b8d3b'
camel_files = ['camel-core/src/main/java/org/apache/camel/processor/validation/SchemaReader.java',
        'camel-core/src/test/java/org/apache/camel/component/validator/ValidatorDtdAccessAbstractTest.java',
        'camel-core/src/test/java/org/apache/camel/component/validator/ValidatorDtdAccessOffTest.java',
        'camel-core/src/test/java/org/apache/camel/component/validator/ValidatorDtdAccessOnTest.java']


In [166]:
activemq_fixing_commit = '9fd5cb7dfe0fcc431f99d5e14206e0090e72f36b'
activemq_files = ['activemq-fileserver/src/main/java/org/apache/activemq/util/FilenameGuardFilter.java',
              'activemq-fileserver/src/main/webapp/index.html',
              'assembly/src/release/conf/jetty.xml'
              ]

In [167]:
restlet_fixing_commit = '97a8d1d62612683817c785e99c4166bcde8cf1c'
restlet_files = ['build/tmpl/text/changes.txt','modules/org.restlet.ext.xml/src/main/java/org/restlet/ext/xml/XmlRepresentation.java']


In [168]:

class vcc:
    
    def __init__(this, target_repo, flags):
        this.repo = git.Repo(target_repo)
        this.flag = flags
    def get_vcccommits(this, commit, files):
    
        blamed_commits = {}
        added_blamed_commits = {}
        removed_blamed_commits = {}
        for file in files:
            
            diff_full_context = this.repo.git.diff("-W", commit + "~", commit, "--", file)
        
            diff_no_context = this.repo.git.diff("-U0", commit + "~", commit, "--", file)
        
            added_lines = set()
           
            removed_lines = set()
        
        for line in diff_full_context.splitlines():
                
                if not line.startswith('@@'):
                    continue
              
                clean_line = line.split('@@')[1]
                half_line = clean_line.split('+')[1]
                start = int(half_line.split(',')[0])
                if ',' in half_line:
                    length = int(half_line.split(',')[1])
                else:
                    length = 1
                 
                for number in range(start, start + length):
                    added_lines.add(number)
            
        for line in diff_no_context.splitlines():
                if not line.startswith('@@'):
                    continue
               
                clean_line = line.split('@@')[1]
                half_line = clean_line.split('+')[0]
                
                half_line = half_line.replace('-','')
                
                start = int(half_line.split(',')[0])
                if ',' in half_line:
                    length = int(half_line.split(',')[1])
                else:
                    length = 1
                
                for number in range(start, start + length):
                    removed_lines.add(number)
                for line in added_lines:
                    if this.flag == '':
                         blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), commit, "--", file)
                    else:
                         blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), this.flag, commit, "--", file)
                    
                         blamed_commit = blame.split(" ")[0]
            
                
                if commit.startswith(blamed_commit):
                    continue
                    

                if blamed_commit in blamed_commits:
                    blamed_commits[blamed_commit] += 1
                else:
                    blamed_commits[blamed_commit] = 1
               
                
                if blamed_commit in added_blamed_commits:
                    added_blamed_commits[blamed_commit] += 1
                else:
                    added_blamed_commits[blamed_commit] = 1
                    
                for line in removed_lines:
                    if this.flag == '':
                        blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), commit + "~", "--", file)
                    else:
                        blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), this.flag, commit + "~", "--", file)
                    
                        blamed_commit = blame.split(" ")[0]
                
               
                if commit.startswith(blamed_commit):
                    continue
            
                if blamed_commit in blamed_commits:
                    blamed_commits[blamed_commit] += 1
                else:
                    blamed_commits[blamed_commit] = 1
                    
                if blamed_commit in removed_blamed_commits:
                    removed_blamed_commits[blamed_commit] += 1
                else:
                    removed_blamed_commits[blamed_commit] = 1
        
        print("Blamed commits for lines which were added: ")
        print(added_blamed_commits)
        print("Blamed commits for lines which were removed: ")
        print(removed_blamed_commits)
        return blamed_commits
    
    def selectvcc(blamed_commits):
        max_blame = 0
        commit = ''
        for key, value in blamed_commits.items():
            if value > max_blame:
                max_blame = value
                commit = key
            
        return commit


#
Finding Vulnerable Contributing Commit and also applying differernt  git blame parameters
FOR THE  FILE: ACTIVEMQ:

In [105]:
for item in ['', '-w', '-wM', '-wC', '-wCC', '-wCCC']:
    # Run the vulnerability_causing commit finder for rest repository with specified flag
    print("rest with: " + item)
    rest_vcc_finder = get_vcccommits(local_link_camel, item)
    result = selectvcc.get_blamed_commits(camel_fixing_commit,camel_files) 
    print("************Total blamed commits*************: ")
    print(result)
    vulnerability_causing_commit = get_vcccommits.select_vulnerability_causing_commit(result)
    print("Vulnerability Causing Commit: ")
    print(vulnerability_causing_commit)

rest with: 
Blamed commits for lines which were added: 
{'115cb3affbe2': 146, '77e079a83861': 47, 'a7d04c7efd46': 2}
Blamed commits for lines which were removed: 
{'115cb3affbe2': 4}
************Total blamed commits*************: 
{'115cb3affbe2': 150, '77e079a83861': 47, 'a7d04c7efd46': 2}
Vulnerability Causing Commit: 
115cb3affbe2
rest with: -w
Blamed commits for lines which were added: 
{'115cb3affbe2': 148, '77e079a83861': 46, 'a7d04c7efd46': 2}
Blamed commits for lines which were removed: 
{'115cb3affbe2': 3, '77e079a83861': 1}
************Total blamed commits*************: 
{'115cb3affbe2': 151, '77e079a83861': 47, 'a7d04c7efd46': 2}
Vulnerability Causing Commit: 
115cb3affbe2
rest with: -wM
Blamed commits for lines which were added: 
{'115cb3affbe2': 148, '77e079a83861': 46, 'a7d04c7efd46': 2}
Blamed commits for lines which were removed: 
{'115cb3affbe2': 3, '77e079a83861': 1}
************Total blamed commits*************: 
{'115cb3affbe2': 151, '77e079a83861': 47, 'a7d04c7efd4

In [106]:
import numpy as np
import pandas as pd
import os
import re
import copy
from collections import Counter
# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
from collections import Counter
import git
from git import RemoteProgress
import pprint as pprint
from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from collections import Counter
%matplotlib inline

In [107]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [108]:
repo = Repo("camel-master")
fixing_commit ="2c6964ae94d8f9a9c9a32e5ae5a0b794e8b8d3b"

In [109]:
VCCCommits=['115cb3affbe2']

In [110]:
show_info = repo.git.show("-s", fixing_commit).splitlines()
for line in show_info:
    print(line)

commit 2c6964ae94d8f9a9c9a32e5ae5a0b794e8b8d3be
Author: Franz Forsthofer <franz.forsthofer@sap.com>
Date:   Fri Feb 24 15:05:36 2017 +0100

    CAMEL-10894: XML Validator: DTD Handling improved


In [111]:
total_affectedfiles = repo.git.show("--stat",fixing_commit).splitlines()
print("the total number of affected files are")
total_affectedfiles=total_affectedfiles[6:]
for line in total_affectedfiles:
    print(line)

the total number of affected files are
 .../camel/processor/validation/SchemaReader.java   | 19 ++++-
 .../validator/ValidatorDtdAccessAbstractTest.java  | 86 ++++++++++++++++++++++
 .../validator/ValidatorDtdAccessOffTest.java       | 61 +++++++++++++++
 .../validator/ValidatorDtdAccessOnTest.java        | 61 +++++++++++++++
 4 files changed, 223 insertions(+), 4 deletions(-)


## Is the of the current fixing commit and its corresponding VCC the same?

In [112]:
for VCC in num_VCCCommits:
    if VCC[0]=='^':
        VCC=VCC[1:]
    try:
        VCCParent= repo.git.show(VCC+"^").splitlines()
    except: 
        pass
print("Parent commit exists, as such it isn't an initial commit!")

Parent commit exists, as such it isn't an initial commit!


##Is the developer of the current fixing commit and its corresponding VCC 
the  same?

In [115]:
commit=repo.commit(fixingCommit)
fixing_Commit_Author=commit.author.email
print(fixing_Commit_Author)
for commits in num_VCCCommits:
    VCCcommit=repo.commit(commits)
    VCCCommitAuthor=VCCcommit.author.email
    if(fixinCommitAuthor==VCCCommitAuthor):
        print("Yes! fixing commit author: {} is same in commit: {}".format(fixinCommitAuthor,VCCcommit))

franz.forsthofer@sap.com
Yes! fixing commit author: franz.forsthofer@sap.com is same in commit: 115cb3affbe2b0a866f7b686035b5b73626b0dca


#How many days were between the current fixing commit and its corresponding VCC?

In [114]:
from datetime import datetime
commit=repo.commit(fixingCommit)
fixing_Commit_Author=commit.author.email
dateof_Fixing=repo.git.show("-s","--format=%ci",commit).split(" ")
dateof_Fixing=dateFixing[0]
dateFixing=datetime.strptime("{}".format(dateFixing), '%Y-%m-%d').date()
time_different = 0
sum = 0
count = 0
for commits in num_VCCCommits:
    date=repo.git.show("-s","--format=%ci",commits).split(" ")
    date=date[0]
    date= datetime.strptime("{}".format(date), '%Y-%m-%d').date()
    days=dateFixing-date
    print("Fixing commit: {} and VCC commit: {} was fixed in: {} ".format(commit,commits,days))
    

Fixing commit: 2c6964ae94d8f9a9c9a32e5ae5a0b794e8b8d3be and VCC commit: 115cb3affbe2 was fixed in: 406 days, 0:00:00 


#
SILARUDDIN SHAIK
ID: A1782580
1. CVE-ID - CVE-2015-1830
2. Project Link1: https://github.com/apache/activemq
3. Vulnerability Contributing  Commit - e6d20f3932

In [160]:
import numpy as np
import pandas as pd
import os

# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

import git
from git import RemoteProgress

from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [161]:
repo = Repo("activemq/")
fixingCommit="e6d20f3932"


 #
    Question'a': What was the message and title of the fixing commit? Was there any mention of fixing a bug or vulnerability?

In [162]:
import pprint as pprint
message_data = repo.git.show("-s", fixingCommit).splitlines()
pprint.pprint(message_data[4])

('    Applied patch from Aleksi Kallio for AMQ-1075 to add a file server for '
 'out of band BlobMessage support')


#Question 'b' How many total files were affected in the fixing commit?

In [165]:
import pprint as pprint
totalFilesAffected = repo.git.show("--stat", "e6d20f3932").splitlines()

pprint.pprint(totalFilesAffected[10])

(' .../java/org/apache/activemq/util/RestFilter.java  | 241 '
 '+++++++++++++++++++++')


#Question 'c' How many total directories were affected in the fixing commit?

In [7]:
totalDirectoriesAffected = repo.git.show("--stat", "e6d20f3932").splitlines()

pprint.pprint("Directories affected: "+totalDirectoriesAffected[8][0:52])

'Directories affected:  activemq-fileserver/pom.xml                        '


#Question'd'How many total lines of code (including comments and blank lines) were deleted?

In [8]:
totalLinesDeletedInc = repo.git.show("--stat", "e6d20f3932").splitlines()

pprint.pprint("Total lines DELETED including comments and blank lines: "+totalLinesDeletedInc[9][33:46])

'Total lines DELETED including comments and blank lines: eGuardFilter.'


#Question'e'How many total lines of code (including comments and blank lines) were added?

In [9]:
totalLinesAddedInc = repo.git.show("--stat", "e6d20f3932").splitlines()

pprint.pprint("Total lines ADDED including comments and blank lines: "+totalLinesAddedInc[9][17:31])

'Total lines ADDED including comments and blank lines: emq/util/Filen'


#Question'f'How many total lines of code (excluding comments and blank lines) were deleted?

In [16]:
totalLinesDeletedExc = repo.git.show("--stat", "--ignore-blank-lines", "e6d20f3932").splitlines()

pprint.pprint("Total lines DELETED excluding comments and blank lines: "+totalLinesDeletedExc[9][15:46])

('Total lines DELETED excluding comments and blank lines: '
 'ivemq/util/FilenameGuardFilter.')


#Question ‘g’ How many total lines of code (excluding comments and blank lines) were added?

In [17]:
totalLinesAddedExc = repo.git.show("--stat", "--ignore-blank-lines", "e6d20f3932").splitlines()

pprint.pprint("Total lines ADDED excluding comments and blank lines: "+totalLinesAddedExc[9][17:31])

'Total lines ADDED excluding comments and blank lines: emq/util/Filen'


#Question'h'How many days were between the current fixing commit and the previous commit of each affected file?

In [166]:
from datetime import datetime
from datetime import date
DateOfFixingCommit = repo.git.log("-1" ,"--format=\"%ai\"", "9fd5cb7dfe0fcc431f99d5e14206e0090e72f36b","--","activemq-fileserver/pom.xml").splitlines()
DateOfFixingCommit=DateOfFixingCommit[0][1:11]
DateOfPrevCommit = repo.git.log("-1" ,"--format=\"%ai\"","-p","9fd5cb7dfe0fcc431f99d5e14206e0090e72f36b","--","activemq-fileserver/pom.xml").splitlines()
DateOfPrevCommit=DateOfPrevCommit[0][1:11]
DateOfPrevCommit= datetime.strptime(DateOfPrevCommit,"%Y-%m-%d")
DateOfFixingCommit= datetime.strptime(DateOfFixingCommit,"%Y-%m-%d")
between_days =(DateOfFixingCommit-DateOfPrevCommit).days
print(between_days,"days")

0 days


#Question I How many time has each affected file of the current fixing commit been modified in the past since their creation (including rename of the file)?

In [22]:
Commits=repo.git.log("--follow","e6d20f3932","--","activemq-fileserver/pom.xml")
print(Commits.count('commit'))

31


Question‘J’ j.	Which developers have modified each affected file since its creation? 

In [23]:
Developers=repo.git.log("--format=\"%aN\"","e6d20f3932","--","activemq-fileserver/pom.xml")

pprint.pprint(Developers)

'"James Strachan"'


Question'k' Which developers have modified each affected file since its creation?
For each developer identified, how many commits have each of them submitted?

In [192]:
best_Developers=repo.git.shortlog("-sn","e6d20f3932","--","activemq-fileserver/pom.xml")
pprint.pprint(best_Developers)

'     1\tJames Strachan'


#finding vcc

In [126]:
class vcc:
    
    def __init__(this, target_repo, flags):
        this.repo = git.Repo(target_repo)
        this.flag = flags
    def get_vcccommits(this, commit, files):
    
        blamed_commits = {}
        added_blamed_commits = {}
        removed_blamed_commits = {}
        for file in files:
            
            diff_full_context = this.repo.git.diff("-W", commit + "~", commit, "--", file)
        
            diff_no_context = this.repo.git.diff("-U0", commit + "~", commit, "--", file)
        
            added_lines = set()
           
            removed_lines = set()
        
        for line in diff_full_context.splitlines():
                
                if not line.startswith('@@'):
                    continue
              
                clean_line = line.split('@@')[1]
                half_line = clean_line.split('+')[1]
                start = int(half_line.split(',')[0])
                if ',' in half_line:
                    length = int(half_line.split(',')[1])
                else:
                    length = 1
                 
                for number in range(start, start + length):
                    added_lines.add(number)
            
        for line in diff_no_context.splitlines():
                if not line.startswith('@@'):
                    continue
               
                clean_line = line.split('@@')[1]
                half_line = clean_line.split('+')[0]
                
                half_line = half_line.replace('-','')
                
                start = int(half_line.split(',')[0])
                if ',' in half_line:
                    length = int(half_line.split(',')[1])
                else:
                    length = 1
                
                for number in range(start, start + length):
                    removed_lines.add(number)
                for line in added_lines:
                    if this.flag == '':
                         blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), commit, "--", file)
                    else:
                         blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), this.flag, commit, "--", file)
                    
                         blamed_commit = blame.split(" ")[0]
            
                
                if commit.startswith(blamed_commit):
                    continue
                    

                if blamed_commit in blamed_commits:
                    blamed_commits[blamed_commit] += 1
                else:
                    blamed_commits[blamed_commit] = 1
               
                
                if blamed_commit in added_blamed_commits:
                    added_blamed_commits[blamed_commit] += 1
                else:
                    added_blamed_commits[blamed_commit] = 1
                    
                for line in removed_lines:
                    if this.flag == '':
                        blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), commit + "~", "--", file)
                    else:
                        blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), this.flag, commit + "~", "--", file)
                    
                        blamed_commit = blame.split(" ")[0]
                
               
                if commit.startswith(blamed_commit):
                    continue
            
                if blamed_commit in blamed_commits:
                    blamed_commits[blamed_commit] += 1
                else:
                    blamed_commits[blamed_commit] = 1
                    
                if blamed_commit in removed_blamed_commits:
                    removed_blamed_commits[blamed_commit] += 1
                else:
                    removed_blamed_commits[blamed_commit] = 1
        
        print("Blamed commits for lines which were added: ")
        print(added_blamed_commits)
        print("Blamed commits for lines which were removed: ")
        print(removed_blamed_commits)
        return blamed_commits
    
    def selectvcc(blamed_commits):
        max_blame = 0
        commit = ''
        for key, value in blamed_commits.items():
            if value > max_blame:
                max_blame = value
                commit = key
            
        return commit


#
Finding Vulnerability Contributing commit 
For Restlet_Framework file.
Applying different types of gitblame paramters

In [127]:
for item in ['', '-w', '-wM', '-wC', '-wCC', '-wCCC']:
    print("rest with: " + item)
    rest_vcc_finder = get_vcccommits(local_link_activemq, item)
    result = selectvcc.get_blamed_commits(activemq_fixing_commit,activemq_files) 
    print("************Total blamed commits*************: ")
    print(result)
    vulnerability_causing_commit = get_vcccommits.select_vulnerability_causing_commit(result)
    print("Vulnerability Causing Commit: ")
    print(vulnerability_causing_commit)

rest with: 
Blamed commits for lines which were added: 
{'e6d20f3932': 18, '8bf987b134': 1, '47b09a2a89': 3, '540dd5c987': 50, '88bdc562f1': 16, '755345cb53': 94, '3a561cc8c7': 1, '46ccaafcfc': 1, '6167c49fc1': 10, '0da02fca9d': 2, '437ea2f6e5': 1, 'e366917efc': 1, 'fdfc5efe5a': 3, '44a7ffa946': 5, '5a7ceacce9': 1, '84c4b9db56': 8, 'd37786ea13': 2, '12585eb5be': 10, '2b019ea627': 1, 'e8a06c43bb': 1, '75c659f122': 1}
Blamed commits for lines which were removed: 
{'540dd5c987': 1, 'e6d20f3932': 2}
************Total blamed commits*************: 
{'e6d20f3932': 20, '8bf987b134': 1, '47b09a2a89': 3, '540dd5c987': 51, '88bdc562f1': 16, '755345cb53': 94, '3a561cc8c7': 1, '46ccaafcfc': 1, '6167c49fc1': 10, '0da02fca9d': 2, '437ea2f6e5': 1, 'e366917efc': 1, 'fdfc5efe5a': 3, '44a7ffa946': 5, '5a7ceacce9': 1, '84c4b9db56': 8, 'd37786ea13': 2, '12585eb5be': 10, '2b019ea627': 1, 'e8a06c43bb': 1, '75c659f122': 1}
Vulnerability Causing Commit: 
755345cb53
rest with: -w
Blamed commits for lines which 

In [182]:
import os
# Set the git executable location. If you are running this script, make sure you set this to be where git is 
# on your computer (My git is in an odd place due to the way my computer is set up!)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git
import datetime
import time

In [183]:
import numpy as np
import pandas as pd
import os
import re
import copy
from collections import Counter
# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
from collections import Counter
import git
from git import RemoteProgress
import pprint as pprint
from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from collections import Counter
%matplotlib inline

In [184]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [185]:
repo = Repo("activemq/")
fixing_commit ="9fd5cb7dfe0fcc431f99d5e14206e0090e72f36b"

In [186]:
show_info = repo.git.show("-s", fixing_commit).splitlines()
for line in show_info:
    print(line)

commit 9fd5cb7dfe0fcc431f99d5e14206e0090e72f36b
Author: Dejan Bosanac <dejan@nighttale.net>
Date:   Thu Apr 30 12:08:26 2015 +0200

    https://issues.apache.org/jira/browse/AMQ-5754 - disable file server by default


In [187]:
total_affectedfiles = repo.git.show("--stat",fixing_commit).splitlines()
print("the total number of affected files are")
total_affectedfiles=total_affectedfiles[6:]
for line in total_affectedfiles:
    print(line)

the total number of affected files are
 .../src/main/java/org/apache/activemq/util/FilenameGuardFilter.java   | 2 +-
 activemq-fileserver/src/main/webapp/index.html                        | 4 ++--
 assembly/src/release/conf/jetty.xml                                   | 3 +++
 3 files changed, 6 insertions(+), 3 deletions(-)


In [188]:
VCCCommits= ['e6d20f3932']

#Is the current fixing commit and its corresponding VCC the same?

In [189]:
for VCC in VCCCommits:
    if VCC[0]=='^':
        VCC=VCC[1:]
    try:
        VCCParent= repo.git.show(VCC+"^").splitlines()
    except: 
        pass
print("Parent commit exists, as such it isn't an initial commit!")

Parent commit exists, as such it isn't an initial commit!


In [190]:
VCCCommits= ['755345cb53']

#Is the developer of the current fixing commit and its corresponding VCC 
the  same?

In [ ]:
commit=repo.commit(fixingCommit)
fixing_Commit_Author=commit.author.email
print(fixing_Commit_Author)
for commits in num_VCCCommits:
    VCCcommit=repo.commit(commits)
    VCCCommitAuthor=VCCcommit.author.email
    if(fixinCommitAuthor==VCCCommitAuthor):
        print("Yes! fixing commit author: {} is same in commit: {}".format(fixinCommitAuthor,VCCcommit))

#How many days were between the current fixing commit and its corresponding VCC?

In [ ]:
from datetime import datetime
commit=repo.commit(fixingCommit)
fixinCommitAuthor=commit.author.email
dateFixing=repo.git.show("-s","--format=%ci",commit).split(" ")
dateFixing=dateFixing[0]
dateFixing=datetime.strptime("{}".format(dateFixing), '%Y-%m-%d').date()
time_different = 0
sum = 0
count = 0
for commits in VCCCommits:
    date=repo.git.show("-s","--format=%ci",commits).split(" ")
    date=date[0]
    date= datetime.strptime("{}".format(date), '%Y-%m-%d').date()
    days=dateFixing-date
    print("Fixing commit: {} and VCC commit: {} was fixed in: {} ".format(commit,commits,days))

#
SILARUDDIN SHAIK
ID: A1782580
1. CVE-ID - CVE-2017-14949
2. Project Link2: https://github.com/restlet/restlet-framework-java
3. Vulnerability Contributing Commit - d1315e7e6e

In [176]:
import numpy as np
import pandas as pd
import os

# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

import git
from git import RemoteProgress

from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [25]:
repo = Repo("restlet-framework-java/")
fixingCommit="d1315e7e6e"

#Question'a': What was the message and title of the fixing commit? Was there any mention of fixing a bug or vulnerability?

In [26]:
import pprint as pprint
message_data = repo.git.show("-s", fixingCommit).splitlines()
pprint.pprint(message_data[4])

'     - Renamed XSLT extension into XML extension.'


#Question 'b' How many total files were affected in the fixing commit?

In [68]:
import pprint as pprint

totalFilesAffected = repo.git.show("--stat", "d1315e7e6e").splitlines()
pprint.pprint(totalFilesAffected[12])

' .../src/org/restlet/ext/xml/DomRepresentation.java | 241 ++++++++'


#Question 'c' How many total directories were affected in the fixing commit?

In [67]:
import pprint as pprint
NumberofDirecAffected = repo.git.show("--stat", "d1315e7e6e").splitlines()

pprint.pprint("Directories affected: "+NumberofDirecAffected[7][20:33])


'Directories affected: .ext.xml/.cla'


#Question'd'How many total lines of code (including comments and blank lines) were deleted?



In [122]:
numberoflinesdeleted = repo.git.show("--stat", "d1315e7e6e").splitlines()

pprint.pprint("Total lines DELETED including comments and blank lines: "+numberoflinesdeleted[10])


('Total lines DELETED including comments and blank lines:  '
 'modules/org.restlet.ext.xml/build.properties       |   4 +')


#Question'e'How many total lines of code (including comments and blank lines) were added?

In [129]:
numberoflinesadded = repo.git.show("--stat", "d1315e7e6e").splitlines()
pprint.pprint("Total lines ADDED including comments and blank lines: "+numberoflinesadded[12][32:51])


'Total lines ADDED including comments and blank lines: Representation.java'


#Question'f'How many total lines of code (excluding comments and blank lines) were deleted?

In [96]:
numberoflinesdeleted = repo.git.show("--stat", "d1315e7e6e").splitlines()

pprint.pprint("Total lines DELETED including comments and blank lines: "+numberoflinesdeleted[18][17:20])


'Total lines DELETED including comments and blank lines: , 2'


#Question ‘g’ How many total lines of code (excluding comments and blank lines) were added?

In [97]:
totalLinesAddedExc = repo.git.show("--stat", "--ignore-blank-lines", "d1315e7e6e").splitlines()

pprint.pprint("Total lines ADDED excluding comments and blank lines: "+totalLinesAddedExc[18][17:20])

'Total lines ADDED excluding comments and blank lines: , 2'


#Question'h'How many days were between the current fixing commit and the previous commit of each affected file?

In [ ]:
from datetime import datetime
from datetime import date
DateOfFixingCommit = repo.git.log("-1" ,"--format=\"%ai\"", "97a8d1d62612683817c785e99c4166bcde8cf1c","--","modules/org.restlet.ext.xml/src/main/java/org/restlet/ext/xml/XmlRepresentation.java").splitlines()
DateOfFixingCommit=DateOfFixingCommit[0][1:11]
DateOfPrevCommit = repo.git.log("-1" ,"--format=\"%ai\"","-p","97a8d1d62612683817c785e99c4166bcde8cf1c^","--","modules/org.restlet.ext.xml/src/main/java/org/restlet/ext/xml/XmlRepresentation.java").splitlines()
DateOfPrevCommit=DateOfPrevCommit[0][1:11]
DateOfPrevCommit= datetime.strptime(DateOfPrevCommit,"%Y-%m-%d")
DateOfFixingCommit= datetime.strptime(DateOfFixingCommit,"%Y-%m-%d")
Days_between=(DateOfFixingCommit-DateOfPrevCommit).days
print(Days_between, "Days")

#Question I How many time has each affected file of the current fixing commit been modified in the past since their creation (including rename of the file)?



In [135]:
Commit_count=repo.git.log("--follow","d1315e7e6e","--","modules/org.restlet.ext.xml/src/main/java/org/restlet/ext/xml/DomRepresentation.java")
print(Commit_count.count('commit'))

0


Question‘J’ j. Which developers have modified each affected file since its creation?

In [137]:
developer_count=repo.git.log("--format=\"%aN\"","d1315e7e6e","--","modules/org.restlet.ext.xml/src/main/java/org/restlet/ext/xml/DomRepresentation.java")
pprint.pprint(developer_count)

''


Question'k' Which developers have modified each affected file since its creation? For each developer identified, how many commits have each of them submitted?

In [138]:
TopDevelopers=repo.git.shortlog("-sn","d1315e7e6e","--","modules/org.restlet.ext.xml/src/main/java/org/restlet/ext/xml/DomRepresentation.java")
pprint.pprint(TopDevelopers)

''


In [ ]:
class vcc:
    
    def __init__(this, target_repo, flags):
        this.repo = git.Repo(target_repo)
        this.flag = flags
    def get_vcccommits(this, commit, files):
    
        blamed_commits = {}
        added_blamed_commits = {}
        removed_blamed_commits = {}
        for file in files:
            
            diff_full_context = this.repo.git.diff("-W", commit + "~", commit, "--", file)
        
            diff_no_context = this.repo.git.diff("-U0", commit + "~", commit, "--", file)
        
            added_lines = set()
           
            removed_lines = set()
        
        for line in diff_full_context.splitlines():
                
                if not line.startswith('@@'):
                    continue
              
                clean_line = line.split('@@')[1]
                half_line = clean_line.split('+')[1]
                start = int(half_line.split(',')[0])
                if ',' in half_line:
                    length = int(half_line.split(',')[1])
                else:
                    length = 1
                 
                for number in range(start, start + length):
                    added_lines.add(number)
            
        for line in diff_no_context.splitlines():
                if not line.startswith('@@'):
                    continue
               
                clean_line = line.split('@@')[1]
                half_line = clean_line.split('+')[0]
                
                half_line = half_line.replace('-','')
                
                start = int(half_line.split(',')[0])
                if ',' in half_line:
                    length = int(half_line.split(',')[1])
                else:
                    length = 1
                
                for number in range(start, start + length):
                    removed_lines.add(number)
                for line in added_lines:
                    if this.flag == '':
                         blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), commit, "--", file)
                    else:
                         blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), this.flag, commit, "--", file)
                    
                         blamed_commit = blame.split(" ")[0]
            
                
                if commit.startswith(blamed_commit):
                    continue
                    

                if blamed_commit in blamed_commits:
                    blamed_commits[blamed_commit] += 1
                else:
                    blamed_commits[blamed_commit] = 1
               
                
                if blamed_commit in added_blamed_commits:
                    added_blamed_commits[blamed_commit] += 1
                else:
                    added_blamed_commits[blamed_commit] = 1
                    
                for line in removed_lines:
                    if this.flag == '':
                        blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), commit + "~", "--", file)
                    else:
                        blame = this.repo.git.blame("-L", str(line) + "," + str(line+1), this.flag, commit + "~", "--", file)
                    
                        blamed_commit = blame.split(" ")[0]
                
               
                if commit.startswith(blamed_commit):
                    continue
            
                if blamed_commit in blamed_commits:
                    blamed_commits[blamed_commit] += 1
                else:
                    blamed_commits[blamed_commit] = 1
                    
                if blamed_commit in removed_blamed_commits:
                    removed_blamed_commits[blamed_commit] += 1
                else:
                    removed_blamed_commits[blamed_commit] = 1
        
        print("Blamed commits for lines which were added: ")
        print(added_blamed_commits)
        print("Blamed commits for lines which were removed: ")
        print(removed_blamed_commits)
        return blamed_commits
    
    def selectvcc(blamed_commits):
        max_blame = 0
        commit = ''
        for key, value in blamed_commits.items():
            if value > max_blame:
                max_blame = value
                commit = key
            
        return commit


In [193]:
for item in ['', '-w', '-wM', '-wC', '-wCC', '-wCCC']:
    # Run the vulnerability_causing commit finder for rest repository with specified flag
    print("rest with: " + item)
    rest_vcc_finder = vulnerability_causing_commit_finder(local_link_restlet_framework, item)
    result = rest_vcc_finder.get_blamed_commits(restlet_fixing_commit,restlet_files) 
    print("************Total blamed commits*************: ")
    print(result)
    vulnerability_causing_commit = vulnerability_causing_commit_finder.select_vulnerability_causing_commit(result)
    print("Vulnerability Causing Commit: ")
    print(vulnerability_causing_commit)

rest with: 
Blamed commits for lines which were added: 
{'a0600a06db': 3, '786320c793': 1, '5e02ce6de2': 1, 'cf52e3dfcc': 1, '405b6f6c83': 802, 'f59b2a81a2': 4, 'bcb665dc02': 4, 'c06f3eec75': 9, 'b356601782': 6, 'a72a92455a': 2, 'a551f961b2': 25, '2d0239f7a4': 17}
Blamed commits for lines which were removed: 
{'405b6f6c83': 1}
************Total blamed commits*************: 
{'a0600a06db': 3, '786320c793': 1, '5e02ce6de2': 1, 'cf52e3dfcc': 1, '405b6f6c83': 803, 'f59b2a81a2': 4, 'bcb665dc02': 4, 'c06f3eec75': 9, 'b356601782': 6, 'a72a92455a': 2, 'a551f961b2': 25, '2d0239f7a4': 17}
Vulnerability Causing Commit: 
405b6f6c83
rest with: -w
Blamed commits for lines which were added: 
{'a0600a06db': 4, '5e02ce6de2': 1, 'cf52e3dfcc': 1, 'd1315e7e6e': 465, 'f59b2a81a2': 4, 'bcb665dc02': 4, 'c3e5155261': 1, '4d2c4ab61a': 64, 'c06f3eec75': 9, 'b356601782': 6, 'a72a92455a': 2, 'b27c2ddcb4': 98, '5d97c80683': 6, 'a551f961b2': 25, '233727c1d0': 87, 'a90348ed5d': 53, '0889cac97a': 2, '1dd4a56df4': 1, 

#
SILARUDDIN SHAIK
ID: A1782580
1. CVE-ID - CVE-2018-8027
2. Project Link1: https://github.com/apache/camel
3. Vulnerability contributing Commit - "115cb3affbe2"

In [226]:
import os
# Set the git executable location. If you are running this script, make sure you set this to be where git is 
# on your computer (My git is in an odd place due to the way my computer is set up!)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git
import datetime
import time

In [227]:
import numpy as np
import pandas as pd
import os
import re
import copy
from collections import Counter
# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
from collections import Counter
import git
from git import RemoteProgress
import pprint as pprint
from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from collections import Counter
%matplotlib inline

In [228]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [229]:
repo = Repo("restlet-framework-java")
fixing_commit ="97a8d1d62612683817c785e99c4166bcde8cf1c"

In [230]:
show_info = repo.git.show("-s", fixing_commit).splitlines()
for line in show_info:
    print(line)

commit 97a8d1d62612683817c785e99c4166bcde8cf1c7
Author: thboileau <thboileau@gmail.com>
Date:   Mon Oct 2 14:31:38 2017 +0200

    XEE injection security in XML extension. Reported by Man Yue Mo.


In [231]:
total_affectedfiles = repo.git.show("--stat",fixing_commit).splitlines()
print("the total number of affected files are")
total_affectedfiles=total_affectedfiles[6:]
for line in total_affectedfiles:
    print(line)

the total number of affected files are
 build/tmpl/text/changes.txt                                          | 3 +++
 .../src/main/java/org/restlet/ext/xml/XmlRepresentation.java         | 5 ++++-
 2 files changed, 7 insertions(+), 1 deletion(-)


In [232]:
VCCCommits=['d1315e7e6e']

#Is the current fixing commit and its corresponding VCC the same??

In [233]:
for VCC in VCCCommits:
    if VCC[0]=='^':
        VCC=VCC[1:]
    try:
        VCCParent= repo.git.show(VCC+"^").splitlines()
    except: 
        pass
print("Parent commit exists, as such it isn't an initial commit!")

Parent commit exists, as such it isn't an initial commit!


#Is the developer of the current fixing commit and its corresponding VCC 
the  same?

In [ ]:
commit=repo.commit(fixingCommit)
fixing_Commit_Author=commit.author.email
print(fixing_Commit_Author)
for commits in num_VCCCommits:
    VCCcommit=repo.commit(commits)
    VCCCommitAuthor=VCCcommit.author.email
    if(fixinCommitAuthor==VCCCommitAuthor):
        print("Yes! fixing commit author: {} is same in commit: {}".format(fixinCommitAuthor,VCCcommit))

#How  many  days  were  between  the  current  fixing  commit  and  its 
corresponding  VCC?

In [ ]:
from datetime import datetime
commit=repo.commit(fixingCommit)
fixinCommitAuthor=commit.author.email
dateFixing=repo.git.show("-s","--format=%ci",commit).split(" ")
dateFixing=dateFixing[0]
dateFixing=datetime.strptime("{}".format(dateFixing), '%Y-%m-%d').date()
time_different = 0
sum = 0
count = 0
for commits in VCCCommits:
    date=repo.git.show("-s","--format=%ci",commits).split(" ")
    date=date[0]
    date= datetime.strptime("{}".format(date), '%Y-%m-%d').date()
    days=dateFixing-date
    print("Fixing commit: {} and VCC commit: {} was fixed in: {} ".format(commit,commits,days))
    

#Question'a': What was the message and title of the fixing commit? Was there any mention of fixing a bug or vulnerability?

In [141]:

import pprint as pprint
show_data = repo.git.show("-s", fixingCommit).splitlines()
pprint.pprint(show_data[4])

'    CAMEL-9517: Validator Endpoint- clearCachedSchema added'


#Question 'b' How many total files were affected in the fixing commit?

In [142]:
numberoffilesaffected = repo.git.show("--stat", "115cb3affbe2").splitlines()
pprint.pprint(numberoffilesaffected[6])

' .../component/validator/ValidatorEndpoint.java     |  74 +++++--'


In [78]:
#Question 'c' How many total directories were affected in the fixing commit?

In [143]:
totalDirectoriesAffected = repo.git.show("--stat", "115cb3affbe2").splitlines()

pprint.pprint("Directories affected: "+totalDirectoriesAffected[8][0:52])

'Directories affected:  .../processor/validation/ValidatingProcessor.java  '


#Question'd'How many total lines of code (including comments and blank lines) were deleted?

In [144]:
totalLinesDeletedInc = repo.git.show("--stat", "115cb3affbe2").splitlines()

pprint.pprint("Total lines DELETED including comments and blank lines: "+totalLinesDeletedInc[10][33:46])

'Total lines DELETED including comments and blank lines: +), 99 deleti'


#Question'e'How many total lines of code (including comments and blank lines) were added?

In [145]:
totalLinesAddedIncluded = repo.git.show("--stat", "115cb3affbe2").splitlines()

pprint.pprint("Total lines ADDED including comments and blank lines: "+totalLinesAddedIncluded[10][17:31])

'Total lines ADDED including comments and blank lines:  493 insertion'


#Question'f'How many total lines of code (excluding comments and blank lines) were deleted?

In [146]:
deleted_lines = repo.git.show("--stat", "--ignore-blank-lines", "115cb3affbe2").splitlines()

pprint.pprint("Total lines DELETED excluding comments and blank lines 7 "+deleted_lines[5][17:31])

'Total lines DELETED excluding comments and blank lines 7 '


#Question ‘g’ How many total lines of code (excluding comments and blank lines) were added?

In [147]:
lines_added = repo.git.show("--stat", "--ignore-blank-lines", "115cb3affbe2").splitlines()

pprint.pprint("Total lines ADDED excluding comments and blank lines: "+lines_added[2][27:31])

'Total lines ADDED excluding comments and blank lines:  201'


#Question'h'How many days were between the current fixing commit and the previous commit of each affected file?

In [150]:
from datetime import datetime
from datetime import date
DateOfFixingCommit = repo.git.log("-1" ,"--format=\"%ai\"", "115cb3affbe2", "--", "camel-core/src/main/java/org/apache/camel/processor/validation/ValidatingProcessor.java").splitlines()
DateOfFixingCommit=DateOfFixingCommit[0][1:11]
DateOfPrevCommit = repo.git.log("-1" ,"--format=\"%ai\"","-p", "115cb3affbe2^", "--", "camel-core/src/main/java/org/apache/camel/processor/validation/ValidatingProcessor.java").splitlines()
DateOfPrevCommit=DateOfPrevCommit[0][1:11]
DateOfPrevCommit= datetime.strptime(DateOfPrevCommit,"%Y-%m-%d")
DateOfFixingCommit= datetime.strptime(DateOfFixingCommit,"%Y-%m-%d")
Between_days=(DateOfFixingCommit-DateOfPrevCommit).days
print(Between_days,"Days")

429 Days


#Question I How many time has each affected file of the current fixing commit been modified in the past since their creation (including rename of the file)?

In [151]:
Commits=repo.git.log("--follow","115cb3affbe2","--","camel-core/src/main/java/org/apache/camel/processor/validation/ValidatingProcessor.java")
print(Commits.count('commit'))

31


Question‘J’ j. Which developers have modified each affected file since its creation?

In [152]:
Developers=repo.git.log("--format=\"%aN\"","115cb3affbe2","--",
"camel-core/src/main/java/org/apache/camel/processor/validation/ValidatingProcessor.java")
pprint.pprint(Developers)

('"Franz Forsthofer"\n'
 '"Claus Ibsen"\n'
 '"Willem Jiang"\n'
 '"Babak Vahdat"\n'
 '"Willem Jiang"\n'
 '"cmueller"\n'
 '"Babak Vahdat"\n'
 '"Claus Ibsen"\n'
 '"Claus Ibsen"\n'
 '"Claus Ibsen"\n'
 '"Willem Ning Jiang"\n'
 '"Claus Ibsen"\n'
 '"Hadrian Zbarcea"\n'
 '"Claus Ibsen"\n'
 '"Claus Ibsen"\n'
 '"Christian Mueller"\n'
 '"Claus Ibsen"\n'
 '"Claus Ibsen"\n'
 '"Claus Ibsen"\n'
 '"Claus Ibsen"\n'
 '"Claus Ibsen"\n'
 '"Roman Kalukiewicz"\n'
 '"Claus Ibsen"\n'
 '"Claus Ibsen"\n'
 '"Claus Ibsen"\n'
 '"Claus Ibsen"\n'
 '"Willem Ning Jiang"\n'
 '"Hiram R. Chirino"\n'
 '"James Strachan"\n'
 '"James Strachan"\n'
 '"James Strachan"')


Question'k' Which developers have modified each affected file since its creation? For each developer identified, how many commits have each of them submitted?

In [153]:
TopDevelopers=repo.git.shortlog("-sn","115cb3affbe2","--",
                                "camel-core/src/main/java/org/apache/camel/processor/validation/ValidatingProcessor.java")
pprint.pprint(TopDevelopers)

('    16\tClaus Ibsen\n'
 '     3\tJames Strachan\n'
 '     2\tWillem Jiang\n'
 '     2\tWillem Ning Jiang\n'
 '     2\tBabak Vahdat\n'
 '     1\tRoman Kalukiewicz\n'
 '     1\tcmueller\n'
 '     1\tChristian Mueller\n'
 '     1\tFranz Forsthofer\n'
 '     1\tHadrian Zbarcea\n'
 '     1\tHiram R. Chirino')
